In [1]:
# Author : Arthur Prigent
# Email: aprigent@geomar.de

In [2]:
from load_librairies import *
import xscale.signal.fitting as xsf

dir_hist_sst = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/historical/tos/atl3/'
dir_hist_uas = '/Volumes/Arthur_disk2/Data/CMIP6/Amon/historical/uas/atl4/'

dir_ssp585_sst = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/ssp585/tos/atl3/'
dir_ssp126_sst = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/ssp126/tos/atl3/'
dir_ssp245_sst = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/ssp245/tos/atl3/'
dir_ssp370_sst = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/ssp370/tos/atl3/'

dir_ssp585_uas = '/Volumes/Arthur_disk2/Data/CMIP6/Amon/ssp585/uas/atl4/'
path_fig = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/figures/'
lon_min = -20
lon_max = 0
lat_min = -3
lat_max = 3

period_str_1 = 1950
period_end_1 = 1999

period_str_2 = 2050
period_end_2 = 2099
now = datetime.now()
date_time = now.strftime("%d/%m/%Y")
def nandetrend(y):
    ''' Remove the linear trend from the data '''
    x = np.arange(0,y.shape[0],1)
    
    not_nan_ind = ~np.isnan(y)
    m, b, r_val, p_val, std_err = stats.linregress(x[not_nan_ind],np.array(y)[not_nan_ind])
    
    
    y_detrended_tmp= np.array(y) - m*x
    
    y_detrended  = xr.Dataset({'detrended': (['time'],y_detrended_tmp)}
                       ,coords={'time':y.time[:]})
    return y_detrended

def is_jja(month):
    return (month >= 6) & (month <= 8)
path_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/scripts/new_version/to_publish/data/'


In [3]:

model = ['ACCESS-CM2',
         'ACCESS-ESM1-5', 'BCC-CSM2-MR',
         'CAMS-CSM1-0', 'CanESM5', 'EC-Earth3', 'EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
         'INM-CM4-8', 'INM-CM5-0',
         'IPSL-CM6A-LR', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0','NESM3']


In [4]:
sst_mean_model_jja_1 = np.ones((len(model)))*np.nan
sst_mean_model_jja_2 = np.ones((len(model)))*np.nan
sst_mean_model_jja_3 = np.ones((len(model)))*np.nan
sst_mean_model_jja_4 = np.ones((len(model)))*np.nan
sst_mean_model_jja_5 = np.ones((len(model)))*np.nan

sst_mean_model_1 = np.ones((len(model)))*np.nan
sst_mean_model_2 = np.ones((len(model)))*np.nan
sst_mean_model_3 = np.ones((len(model)))*np.nan
sst_mean_model_4 = np.ones((len(model)))*np.nan
sst_mean_model_5 = np.ones((len(model)))*np.nan


for i in range(len(model)):
    ## Historical ##
    try:
        nc_tmp = xr.open_dataset(dir_hist_sst+'tos_Omon_'+model[i]+'_historical_r1i1p1f1_187001-201412_1deg_atl3.nc')
        print(model[i])
        sst_tmp_1 = nc_tmp.tos[:] 

        try:

            sst_tmp_1 = sst_tmp_1.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
        except TypeError:

            sst_tmp_1['time'] = sst_tmp_1.indexes['time'].to_datetimeindex()
            sst_tmp_1 = sst_tmp_1.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))


        sst_tmp_1 = sst_tmp_1.mean(dim='lon').mean(dim='lat')
        sst_jja_mean = sst_tmp_1.sel(time=is_jja(sst_tmp_1['time.month'])).mean(dim='time')
        sst_mean = sst_tmp_1.mean(dim='time')

        sst_mean_model_jja_1[i] = sst_jja_mean.values
        sst_mean_model_1[i] = sst_mean.values

    
    except FileNotFoundError:
        print('No file, pass')
        sst_mean_model_jja_1[i] = np.nan
        sst_mean_model_1[i] = np.nan
    
    ## RCP85 ##
    try:
        nc_tmp = xr.open_dataset(dir_ssp585_sst+'tos_Omon_'+model[i]+'_ssp585_r1i1p1f1_201501-209912_1deg_atl3.nc')
        print(model[i])
        sst_tmp_2 = nc_tmp.tos[:]
        try:

            sst_tmp_2 = sst_tmp_2.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
        except TypeError:

            sst_tmp_2['time'] = sst_tmp_2.indexes['time'].to_datetimeindex()
            sst_tmp_2 = sst_tmp_2.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))


        sst_tmp_2 = sst_tmp_2.mean(dim='lon').mean(dim='lat')
        sst_jja_mean = sst_tmp_2.sel(time=is_jja(sst_tmp_2['time.month'])).mean(dim='time')
        sst_mean = sst_tmp_2.mean(dim='time')

        sst_mean_model_jja_2[i] = sst_jja_mean.values
        sst_mean_model_2[i] = sst_mean.values
    except FileNotFoundError:
        print('No file, pass')
        sst_mean_model_jja_2[i] = np.nan
        sst_mean_model_2[i] = np.nan
    
    
    
    ## RCP45 ##
    try:
        nc_tmp = xr.open_dataset(dir_ssp126_sst+'tos_Omon_'+model[i]+'_ssp126_r1i1p1f1_201501-209912_1deg_atl3.nc')
        print(model[i])
        sst_tmp_3 = nc_tmp.tos[:]
        try:

            sst_tmp_3 = sst_tmp_3.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
        except TypeError:

            sst_tmp_3['time'] = sst_tmp_3.indexes['time'].to_datetimeindex()
            sst_tmp_3 = sst_tmp_3.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))


        sst_tmp_3 = sst_tmp_3.mean(dim='lon').mean(dim='lat')
        sst_jja_mean = sst_tmp_3.sel(time=is_jja(sst_tmp_3['time.month'])).mean(dim='time')
        sst_mean = sst_tmp_3.mean(dim='time')

        sst_mean_model_jja_3[i] = sst_jja_mean.values
        sst_mean_model_3[i] = sst_mean.values

    except FileNotFoundError:
        print('No file, pass')
        sst_mean_model_jja_3[i] = np.nan
        sst_mean_model_3[i] = np.nan
    
    ## SSP245 ##
    try:
        nc_tmp = xr.open_dataset(dir_ssp245_sst+'tos_Omon_'+model[i]+'_ssp245_r1i1p1f1_201501-209912_1deg_atl3.nc')
        print(model[i])
        sst_tmp_4 = nc_tmp.tos[:]
        try:

            sst_tmp_4 = sst_tmp_4.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
        except TypeError:

            sst_tmp_4['time'] = sst_tmp_4.indexes['time'].to_datetimeindex()
            sst_tmp_4 = sst_tmp_4.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))


        sst_tmp_4 = sst_tmp_4.mean(dim='lon').mean(dim='lat')
        sst_jja_mean = sst_tmp_4.sel(time=is_jja(sst_tmp_4['time.month'])).mean(dim='time')
        sst_mean = sst_tmp_4.mean(dim='time')

        sst_mean_model_jja_4[i] = sst_jja_mean.values
        sst_mean_model_4[i] = sst_mean.values

    except FileNotFoundError:
        print('No file, pass')
        sst_mean_model_jja_4[i] = np.nan
        sst_mean_model_4[i] = np.nan
    
    
    ## SSP370 ##
    try:
        nc_tmp = xr.open_dataset(dir_ssp370_sst+'tos_Omon_'+model[i]+'_ssp370_r1i1p1f1_201501-209912_1deg_atl3.nc')
        print(model[i])
        sst_tmp_5 = nc_tmp.tos[:]
        try:

            sst_tmp_5 = sst_tmp_5.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
        except TypeError:

            sst_tmp_5['time'] = sst_tmp_5.indexes['time'].to_datetimeindex()
            sst_tmp_5 = sst_tmp_5.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))


        sst_tmp_5 = sst_tmp_5.mean(dim='lon').mean(dim='lat')
        sst_jja_mean = sst_tmp_5.sel(time=is_jja(sst_tmp_5['time.month'])).mean(dim='time')
        sst_mean = sst_tmp_5.mean(dim='time')

        sst_mean_model_jja_5[i] = sst_jja_mean.values
        sst_mean_model_5[i] = sst_mean.values

    except FileNotFoundError:
        print('No file, pass')
        sst_mean_model_jja_5[i] = np.nan
        sst_mean_model_5[i] = np.nan
    

ACCESS-CM2
ACCESS-CM2
ACCESS-CM2
ACCESS-CM2
ACCESS-CM2
ACCESS-ESM1-5
ACCESS-ESM1-5
ACCESS-ESM1-5
ACCESS-ESM1-5
ACCESS-ESM1-5
BCC-CSM2-MR
BCC-CSM2-MR
BCC-CSM2-MR
BCC-CSM2-MR


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in o

BCC-CSM2-MR
CAMS-CSM1-0
CAMS-CSM1-0
CAMS-CSM1-0
CAMS-CSM1-0


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in o

CAMS-CSM1-0
CanESM5
CanESM5
CanESM5


...

/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:106: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:133: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in

INM-CM4-8
INM-CM5-0
INM-CM5-0


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:106: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in 

INM-CM5-0
INM-CM5-0
INM-CM5-0
IPSL-CM6A-LR
IPSL-CM6A-LR
IPSL-CM6A-LR
IPSL-CM6A-LR
No file, pass
MIROC6
MIROC6
MIROC6
MIROC6
MIROC6
MPI-ESM1-2-HR
MPI-ESM1-2-HR
MPI-ESM1-2-HR
MPI-ESM1-2-HR
MPI-ESM1-2-HR
MPI-ESM1-2-LR
MPI-ESM1-2-LR
MPI-ESM1-2-LR
MPI-ESM1-2-LR
MPI-ESM1-2-LR
MRI-ESM2-0
MRI-ESM2-0
MRI-ESM2-0
MRI-ESM2-0
No file, pass
NESM3
NESM3
NESM3
NESM3
No file, pass


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in o

In [5]:

monthly_sst_mean_model_1  = xr.Dataset({'jja_sst': (['model'],sst_mean_model_jja_1),
                                       'sst': (['model'],sst_mean_model_1)}
                           ,coords={'model': model}

                           ,attrs={'standard_name': 'sst',
                                    'long_name': 'Mean SST',
                                    'units': 'K',
                                    'model': 'CMIP5',
                                    'Scenario': 'Historical',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})

monthly_sst_mean_model_2  = xr.Dataset({'jja_sst': (['model'],sst_mean_model_jja_2),
                                       'sst': (['model'],sst_mean_model_2)}
                           ,coords={'model': model}

                           ,attrs={'standard_name': 'sst',
                                    'long_name': 'Mean SST',
                                    'units': 'K',
                                    'model': 'CMIP5',
                                    'Scenario': 'SSP585',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})


monthly_sst_mean_model_3  = xr.Dataset({'jja_sst': (['model'],sst_mean_model_jja_3),
                                       'sst': (['model'],sst_mean_model_3)}
                           ,coords={'model': model}

                           ,attrs={'standard_name': 'sst',
                                    'long_name': 'Mean SST',
                                    'units': 'K',
                                    'model': 'CMIP5',
                                    'Scenario': 'SSP126',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})

monthly_sst_mean_model_4  = xr.Dataset({'jja_sst': (['model'],sst_mean_model_jja_4),
                                       'sst': (['model'],sst_mean_model_4)}
                           ,coords={'model': model}

                           ,attrs={'standard_name': 'sst',
                                    'long_name': 'Mean SST',
                                    'units': 'K',
                                    'model': 'CMIP5',
                                    'Scenario': 'SSP245',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})


monthly_sst_mean_model_5  = xr.Dataset({'jja_sst': (['model'],sst_mean_model_jja_5),
                                       'sst': (['model'],sst_mean_model_5)}
                           ,coords={'model': model}

                           ,attrs={'standard_name': 'sst',
                                    'long_name': 'Mean SST',
                                    'units': 'K',
                                    'model': 'CMIP5',
                                    'Scenario': 'SSP370',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})


In [6]:
monthly_sst_mean_model_1.to_netcdf(path_data+'mean_sst_hist_monthly_CMIP6.nc')
monthly_sst_mean_model_2.to_netcdf(path_data+'mean_sst_ssp585_monthly_CMIP6.nc')
monthly_sst_mean_model_3.to_netcdf(path_data+'mean_sst_ssp126_monthly_CMIP6.nc')
monthly_sst_mean_model_4.to_netcdf(path_data+'mean_sst_ssp245_monthly_CMIP6.nc')
monthly_sst_mean_model_5.to_netcdf(path_data+'mean_sst_ssp370_monthly_CMIP6.nc')
